## Building Your First Multi-Agent AI App with AutoGen

<a target="_blank" href="#">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


Some tasks often benefit from multiple distinct experts that can be abstracted as individual agents. Consider that our task is build an art exhibition director i.e., given a theme and the number of expected art pieces, they can extend the initial theme, suggest a list of relevant styles and artists, create sample images based on a detailed conceptualization  of each piece, and finally generate a catalog of the exhibition.

In theory, we can create a general purpose assistant agent and give it access to many skills that cover a range of expert areas. However, this can make it harder to reason about and debug the agents behavior and also introduces additional challenges as the LLM has to select the right tool or function call at each step. 

To simplify the process of solving these types of tasks, we can create a team of agents, each with specific expertise. For example, we can create an `ArtPlanner` that extends the theme into a detailed plan, an `ArtStyleSelector` that suggests relevant styles and artists, an `ArtConceptualizer` that creates detailed conceptual descriptions, and an `ArtCataloguer` that generates the exhibition catalog. Each agent can be configured with the appropriate tools and LLMs to solve their specific subtask.

In [2]:
import os
llm_config = {"config_list": [{
        "model": "gpt-4-turbo-preview",
        "api_key": os.environ["OPENAI_API_KEY"],  # Note, add your own API key here
    }],
    "temperature": 0.9, 
    "cache_seed": 41,
}

from typing import List
import uuid
import requests
from pathlib import Path
from openai import OpenAI

def generate_and_save_images(query: str, image_size: str = "1024x1024") -> List[str]:
    client = OpenAI()
    response = client.images.generate(model="dall-e-3", prompt=query, n=1, size=image_size)
    saved_files = []
    if response.data:
        for image_data in response.data:
            file_name = str(uuid.uuid4()) + ".png" 
            file_path = Path("data") / file_name
            img_url = image_data.url
            img_response = requests.get(img_url)
            if img_response.status_code == 200:
                with open(file_path, "wb") as img_file:
                    img_file.write(img_response.content)
                    saved_files.append(str(file_path))
                    print(f"Image generated and saved to {file_path}")
    return saved_files 

## Defining a Basic Agent Workflow with Two Agents 

In the following section we will define an agent workflow with only two agents. So what is the orchestation behavior i.e. how the agents interact or decide when to act? Well, this is fairly simple as they will reply to each others messages until the task is completed.

In [3]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager,register_function

# To simplify the process of solving these types of tasks, we can create a team of agents, each with specific expertise. For example, we can create an `ArtPlanner` that extends the theme into a detailed plan, an `ArtStyleSelector` that suggests relevant styles and artists, an `ArtConceptualizer` that creates detailed conceptual descriptions, and an `ArtCataloguer` that generates the exhibition catalog. Each agent can be configured with the appropriate tools and LLMs to solve their specific subtask.

art_director = AssistantAgent(
    name= "art_director", llm_config=llm_config,
    system_message="You are a helpful assistant that is a highly skilled art director. Given a general description of an art project or theme, your goal is to provide a detailed and creative plan that develops the provided description into a very high quality art exhibition. Your plan should include details on the intended location, the layout of the exhibition, the experience of the users as they engage etc.")

style_director = AssistantAgent(
    name= "style_director", llm_config=llm_config,
    system_message="You are a helpful assistant that is a highly skilled art style director. Given a general description of an exhibition plan provided by the art director, your goal is to design a well thought out list of 5  art styles and artists that would be relevant to the exhibition. Each style should be accompanied by a detailed rationale and grounded in the context of the exhibition plan e.g. via location, history etc.")

conceptualizer = AssistantAgent(
    name= "conceptualizer", llm_config=llm_config,
    system_message="You are a helpful assistant that is a highly skilled art conceptualizer. Given a general description of an exhibition plan and a list of art styles and artists provided by the style director, your goal is to create highly detailed conceptual descriptions for 5 art pieces that will be included in the exhibition. Each description should be unique and should be grounded in the context of the exhibition plan and the selected art styles. Each art piece should have a title, a detailed description and some aspirational goals for the viewer experience. Finally, you should generate an image of each art piece.")

cataloguer = AssistantAgent(
    name= "cataloguer", llm_config=llm_config,
    system_message="You are a helpful assistant that is a highly skilled art cataloguer. Given a general description of an exhibition plan, a list of art styles and artists and detailed conceptual descriptions for 5 art pieces provided by the conceptualizer, your goal is to create a detailed exhibition catalog that describes the exhibition, the art pieces, the artists and the styles. Your catalog should be detailed, well written and should be suitable for publication. It should include a cover page, a table of contents, a detailed description of the exhibition, pictures of each of the art pieces, and the styles. When all of this is completed, save the catalog as a well formatted and designed PDF file.")


user_proxy = UserProxyAgent(
    name = "user_proxy", 
    code_execution_config={"work_dir": 
                           "data/scratch", 
                           "use_docker": False},
    human_input_mode="NEVER",
    is_termination_msg = lambda x: x.get("content", "").rstrip().endswith("TERMINATE") if x.get("content") else False, 
    )  

# give the conceptualizer the ability to generate images
register_function(
    generate_and_save_images,
    caller=conceptualizer,  # The assistant agent can suggest calls to the  function
    executor=user_proxy,  # The user proxy agent can execute the function call.
    name="generate_and_save_images",  # By default, the function name is used as the tool name.
    description="Generate images using DALL-E",
)

groupchat = GroupChat(agents=[art_director, style_director, conceptualizer, cataloguer, user_proxy], messages=[], speaker_selection_method="auto" )
groupchat_manager =  GroupChatManager(groupchat=groupchat, llm_config=llm_config

/opt/homebrew/Caskroom/miniconda/base/envs/autog/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
task = " I'd like to organize an art exhibition on the theme of 'The Future of Humanity'. I am originally from New Orleans but currently living in San Francisco where the exhibition is to be held" 

results = user_proxy.initiate_chat(groupchat_manager, message=task)

user_proxy (to chat_manager):

 I'd like to organize an art exhibition on the theme of 'The Future of Humanity'. I am originally from New Orleans but currently living in San Francisco where the exhibition is to be held

--------------------------------------------------------------------------------
art_director (to chat_manager):

Title: "Beyond Horizons: The Future of Humanity"

Location: Exploratorium, San Francisco

Concept Overview:
"Beyond Horizons: The Future of Humanity" is an immersive art exhibition designed to engage and challenge visitors' perceptions of the future. Set within the innovative space of the Exploratorium in San Francisco, this exhibition combines interactive installations, digital art, and traditional mediums to explore potential futures of humanity through the lenses of technology, environmental sustainability, societal shifts, and speculative evolution. The exhibit seeks not only to present artistic interpretations but also to foster dialogue and introspecti